In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd
%cd "/content/"
!pwd

/content
/content
/content


Data Loader

In [ ]:
import torch.utils.data as data

from PIL import Image
import os
import os.path
import sys
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset, DataLoader, Dataset
import cv2
from sklearn import preprocessing
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt


In [ ]:
image_ids = []
xs = []
ys = []
zs = []
c = 0
for dir, sub_dir, f in os.walk("/content/drive/My Drive/FYP/Dataset/entire/openEDS2020-GazePrediction/train/sequences"):
  if len(f) > 0:
    c += 1
    f = sorted(f)
    seq_id = dir.split("/")[-1]
    temp_images_ids = [seq_id+'_'+x for x in f]
    image_ids.extend(temp_images_ids)
    with open("/content/drive/My Drive/FYP/Dataset/entire/openEDS2020-GazePrediction/train/labels/"+seq_id+".txt", mode = "r") as file:
      labels = file.read()
      labels = labels.split('\n')
      for line in labels:
        line = line.strip('\n')
        if len(line.split(',')) == 4:
          file_reqd, temp_x, temp_y, temp_z = line.split(',') 
          if file_reqd+'.png' in f:
            xs.append(float(temp_x))
            ys.append(float(temp_y))
            zs.append(float(temp_z))
  if(c == 15):
      break

    
import pandas as pd

df = pd.DataFrame({"unique_id":image_ids,'x':xs,'y':ys,'z':zs})
df.head()


,unique_id,x,y,z
0,6679_000.png,-0.133114,-0.021778,0.990861
1,6679_001.png,-0.127335,-0.020104,0.991656
2,6679_002.png,-0.133111,-0.022769,0.990839
3,6679_003.png,-0.127335,-0.020104,0.991656
4,6679_005.png,-0.126190,-0.019146,0.991821


In [ ]:
image_ids = []
xs = []
ys = []
zs = []
c = 0
for dir, sub_dir, f in os.walk("/content/drive/My Drive/FYP/Dataset/entire/openEDS2020-GazePrediction/validation/sequences"):
  if len(f) > 0:
    c += 1
    f = sorted(f)
    seq_id = dir.split("/")[-1]
    temp_images_ids = [seq_id+'_'+x for x in f]
    image_ids.extend(temp_images_ids)
    with open("/content/drive/My Drive/FYP/Dataset/entire/openEDS2020-GazePrediction/validation/labels/"+seq_id+".txt", mode = "r") as file:
      labels = file.read()
      labels = labels.split('\n')
      for line in labels:
        line = line.strip('\n')
        if len(line.split(',')) == 4:
          file_reqd, temp_x, temp_y, temp_z = line.split(',') 
          if file_reqd+'.png' in f:
            xs.append(float(temp_x))
            ys.append(float(temp_y))
            zs.append(float(temp_z))
  if(c == 15):
      break

    
import pandas as pd

df1 = pd.DataFrame({"unique_id":image_ids,'x':xs,'y':ys,'z':zs})
df1.head()


,unique_id,x,y,z
0,7960_000.png,0.204384,0.111141,0.972561
1,7960_001.png,0.203746,0.110971,0.972714
2,7960_002.png,0.202203,0.110214,0.973122
3,7960_003.png,0.201722,0.110209,0.973223
4,7960_004.png,0.201057,0.110906,0.973281


In [ ]:
image_ids = []

c = 0
for dir, sub_dir, f in os.walk("/content/drive/My Drive/FYP/Dataset/entire/openEDS2020-GazePrediction/test/sequences"):
  if len(f) > 0:
    c += 1
    f = sorted(f)
    seq_id = dir.split("/")[-1]
    temp_images_ids = [seq_id+'_'+x for x in f]
    image_ids.extend(temp_images_ids)

  if (c == 50):
    break

import pandas as pd

df2 = pd.DataFrame({"unique_id":image_ids})
df2.head()


,unique_id
0,5401_000.png
1,5401_001.png
2,5401_002.png
3,5401_003.png
4,5401_004.png


In [ ]:
df2

,unique_id
0,5401_000.png
1,5401_001.png
2,5401_002.png
3,5401_003.png
4,5401_004.png
...,...
1361,5449_044.png
1362,5449_045.png
1363,5449_046.png
1364,5449_047.png


In [ ]:
class EyeGazeData(Dataset):
    def __init__(self,data_df,data_dir = './',transform=None,train=True):
        super().__init__()
        self.data_df = data_df
        self.data_dir = data_dir
        self.transform = transform
        self.train = train
    
    def __len__(self):
        return self.data_df.shape[0]
    
    def __getitem__(self,item):
        if self.train:
          uid,x,y,z = self.data_df.iloc[item]
        else:
          uid = self.data_df.iloc[item]['unique_id']
        #uid,x,y,z = self.data_df.iloc[item]
        seq_id,filename = uid.split('_')
        img_path = os.path.join(self.data_dir+'/'+seq_id,filename)
        img = cv2.imread(img_path,1)
        if self.transform is not None:
            img = self.transform(img)
        if self.train:
          return {
              'img' : img,
              'label_x' : torch.tensor(x),
              'label_y' : torch.tensor(y),
              'label_z' : torch.tensor(z),
                }
        else: 
          return {'img':img}
        
        

In [ ]:
data_transform = transforms.Compose([transforms.ToPILImage(),
                                     transforms.Resize(299), 
                                     transforms.CenterCrop(299), 
                                     transforms.ToTensor(), 
                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

train_dataset = EyeGazeData(df, '/content/drive/My Drive/FYP/Dataset/entire/openEDS2020-GazePrediction/train/sequences',data_transform)
train_loader = DataLoader(train_dataset, batch_size=32, num_workers=0, shuffle=True)

val_dataset = EyeGazeData(df1, '/content/drive/My Drive/FYP/Dataset/entire/openEDS2020-GazePrediction/validation/sequences',data_transform)
val_loader = DataLoader(val_dataset, batch_size=32, num_workers=0, shuffle=True)

test_dataset = EyeGazeData(df2, '/content/drive/My Drive/FYP/Dataset/entire/openEDS2020-GazePrediction/test/sequences',data_transform,train=False)
test_loader = DataLoader(val_dataset, batch_size=32, num_workers=0, shuffle=True)



In [ ]:
test_dataset.__getitem__(1)

{'img': tensor([[[-1.6213, -1.6213, -1.6213,  ..., -1.9295, -1.9467, -1.9467],
          [-0.5424, -0.5424, -0.5424,  ..., -1.2788, -1.2788, -1.3302],
          [-0.5938, -0.5938, -0.6109,  ..., -1.0390, -1.0390, -1.0733],
          ...,
          [-0.8164, -0.8164, -0.7993,  ..., -1.7583, -1.7583, -1.7754],
          [-0.8335, -0.8164, -0.7650,  ..., -1.7583, -1.7754, -1.7754],
          [-0.8507, -0.8164, -0.7822,  ..., -1.7583, -1.7754, -1.7583]],
 
         [[-1.5280, -1.5280, -1.5280,  ..., -1.8431, -1.8606, -1.8606],
          [-0.4251, -0.4251, -0.4251,  ..., -1.1779, -1.1779, -1.2304],
          [-0.4776, -0.4776, -0.4951,  ..., -0.9328, -0.9328, -0.9678],
          ...,
          [-0.7052, -0.7052, -0.6877,  ..., -1.6681, -1.6681, -1.6856],
          [-0.7227, -0.7052, -0.6527,  ..., -1.6681, -1.6856, -1.6856],
          [-0.7402, -0.7052, -0.6702,  ..., -1.6681, -1.6856, -1.6681]],
 
         [[-1.2990, -1.2990, -1.2990,  ..., -1.6127, -1.6302, -1.6302],
          [-0.2010, -

Regression Training

In [ ]:
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image
from google.cloud import storage


def get_off_accuracy_regression(model, data_loader, GPU= 0):
    p=0
    freq_pos = np.zeros(81)
    freq_neg = np.zeros(80)
    for img, label in data_loader:
        out = model(img)
        #out = out.int()
        label = label.cuda(GPU)

        for i in range(0,len(label)):
            diff = label[i]+1 - out[i].int()
            if diff>=0:
                freq_pos[diff] +=1
            else:
                freq_neg[diff] +=1

    diffs = []
    for n in range(-80, 81):
        diffs.append(n)
    
    freq_total = np.concatenate((freq_neg, freq_pos))
    freq_total = freq_total/freq_total.sum()
    plt.bar(diffs[50:110], freq_total[50:110])
    plt.title("Distribution of Actual-Prediction")
    plt.xlabel("difference value")
    plt.ylabel("prob")
    print("+/- 1 year accuracy: {:.2f}%".format(freq_total[79:81].sum()*100))
    print("+/- 5 years accuracy: {:.2f}%".format(freq_total[75:86].sum()*100))
    print("+/- 10 years accuracy: {:.2f}%".format(freq_total[70:91].sum()*100))
    
    return
    
  
def evaluate_regression(data_eval, net, criterion, batch_size =32, GPU = 0):
    total_epoch = 0
    total_loss = 0
    for inputs, labels in data_eval:
        outputs = net(inputs.cuda(GPU))
        loss = criterion(outputs.cuda(GPU), labels.float().cuda(GPU))
        total_loss += loss.item()
        total_epoch += len(labels)
    return float(total_loss)/(total_epoch)
  
    
def train_regression(train_loader, val_loader, model, pretrained = False, checkpoint = None, model_name = 'VGG', batch_size=32, learning_rate=5e-05, n_epochs=30, GPU= 0):
    
    
    # Fixed PyTorch random seed for reproducible result
    from torch.autograd import Variable
    torch.manual_seed(1000)
    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    
    pretrained_epoch = 0
    train_losses, valid_losses, train_acc, val_acc = [], [], [], []
    if pretrained == True:
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        pretrained_epoch = checkpoint['epoch']
        train_losses = checkpoint['train_loss']
        valid_losses = checkpoint['valid_loss']
        model.train()
        print("resuming training after epoch: ", pretrained_epoch)
    
    # optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    # Train the network
    # Loop over the data iterator and sample a new batch of training data
    # Get the output from the network, and optimize our loss function.
    
    valid_loss_min = np.Inf
    start_time = time.time()
    for epoch in range(n_epochs):
        train_loss = 0.0
        valid_loss = 0.0
        
        model.train()
        for images in train_loader:
            data = images['img'].squeeze(0).to(device)
            # data = data.squeeze(0)
            targetx = images['label_x'].to(device)
            targety = images['label_y'].to(device)
            targetz = images['label_z'].to(device)
            #clear the gradients of all optimized variables
            optimizer.zero_grad()
            #forward pass the model
            output = model(data)
            #backward pass the model
            #loss = criterion(output,targetx.long())
            lossx = criterion(output,targetx.long())
            lossy = criterion(output,targety.long())
            lossz = criterion(output,targetz.long())
            loss = lossx + lossy + lossz
            loss.backward()
            #Perform a single optimization step
            optimizer.step()
            train_loss += loss.item()*data.size(0)

        model.eval()
        for images in val_loader:
            data = images['img'].squeeze(0).to(device)
            targetx = images['label_x'].to(device)
            targety = images['label_y'].to(device)
            targetz = images['label_z'].to(device)
            target = targetx + targety + targetz
            #forward pass now
            output = model(data)
            #calculate the branch loss
            loss = criterion(output, target.long())
            #update average validation loss
            valid_loss += loss.item()*data.size(0)
        
        train_loss /= len(train_loader.sampler)
        valid_loss /= len(val_loader.sampler)
        
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)
        
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, train_loss, valid_loss))
        
        if valid_loss <= valid_loss_min:
            print("Validation Loss decreased {:0.6f} -> {:0.6f}".format(valid_loss_min,valid_loss))
            valid_loss_min = valid_loss
            torch.save(model.state_dict(), 'best_model_so_far.pth')
    
    
    # plotting
    plt.title("Loss curves w/ lr={}, batch size = {}".format(learning_rate, batch_size))
    plt.axis([0, n_epochs, 0, max(train_losses[0], valid_losses[0])])
    plt.plot(train_losses, label="Train loss")
    plt.plot(valid_losses, label="Validation loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.show()

    print("Final Training Loss: {}".format(train_losses[-1]))
    print("Final Validation Loss: {}".format(valid_losses[-1]))

    print('Finished Training')
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))
    
    return

def regression_demo(data, model):
    k = 0
    for img, label in data:
        #img = img / 2 + 0.5
        #plt.subplot(3, 5, k+1)
        #plt.axis('off')
        pred = model(img)
        for count, i in enumerate(img, 0):
            print(count)
            i = np.transpose(i, [1,2,0])
            plt.imshow(i)
            plt.show()
            print("Actual: ", label[count], " Prediction: ", pred[count])
            k += 1
        #print(label, vgg_regression(img))
            if k > 10:
                break

        break
    
    return

Inception

In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

inception = models.inception_v3(pretrained=True)

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth


In [ ]:
inception.cuda()
inception.eval()

In [ ]:
class inception_regression(nn.Module):
    def __init__(self):
        super(inception_regression, self).__init__()
        self.name = "inception_regression"
        self.fc1 = nn.Linear(1000, 1) #x
        #self.fc2 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(1000, 1) #y
        self.fc3 = nn.Linear(1000, 1) #z
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = x.cuda()
        x = inception(x)
        x = x.view(x.shape[0],-1)
        label1 = self.fc1(x)
        label2= torch.sigmoid(self.fc2(x))  
        label3= self.fc3(x)
        return {'label_x': label1, 'label_y': label2, 'label_z': label3}

        '''x.view(x.shape[0],-1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        x = x.squeeze(1)

        y = y.cuda()
        y = inception(y)
        y = y.view(y.shape[0],-1)
        y = F.relu(self.fc1(y))
        y = self.dropout(y)
        y = F.relu(self.fc2(y))
        y = self.dropout(y)
        y = self.fc3(y)
        y = y.squeeze(1)

        z = z.cuda()
        z = inception(z)
        z = z.view(z.shape[0],-1)
        z = F.relu(self.fc1(z))
        z = self.dropout(z)
        z = F.relu(self.fc2(z))
        z = self.dropout(z)
        z = self.fc3(z)
        z = z.squeeze(1)'''

        #return x

In [ ]:
inc_regression = inception_regression()
inc_regression.cuda()

inception_regression(
  (fc1): Linear(in_features=1000, out_features=1, bias=True)
  (fc2): Linear(in_features=1000, out_features=1, bias=True)
  (fc3): Linear(in_features=1000, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [ ]:
train_regression(train_loader, val_loader, inc_regression, model_name = 'INCEPTION', batch_size = 32, n_epochs = 10)

RuntimeError: ignored